<a href="https://colab.research.google.com/github/MichaelSeitz98/Seminararbeit_Kleinkinderkennung/blob/main/schwellenwertanalyse_mit_ordnereinteilung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Verbindung zu Google Drive herstellen. 

Für Zugriff auf Datensatz,  zum Laden der Modelle und Speichern der untersuchten Bilder. 



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Installation Tensorflow Object Detection API




Da die Tensorflow Object Detection API nicht auf Colab vorinstalliert ist. Muss diese bei jeder neuen Laufzeit neu installiert werden.
Dies geschieht in meheren Schritten. 

Zuerst wird Tensorflow installiert. Aufgrund verschiedener Installationsprobleme hat sich die Version 2.6.0 für mich als zuverlässigste herausgestellt. 

Wenn noch nicht vorhanden, dann werden (via git clone) das tensorflow/models repository von Github geklont. Einiger der Dateien daraus werden für die tatsächliche Installation benötigt (im nächsten Schritt).  

In [ ]:
!pip install tensorflow=="2.6.0"

In [ ]:
import pathlib
import os
 
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Da eine parallele Überwachung von Trainingsmetriken (loss) und Validierungs-Metriken (loss, map, usw.) nicht über colab möglich ist, evaluiere ich das Modell nach abgeschlossenem Trainingslauf. Dazu müssen zwei Anpassungen an dieser Stelle vorgenommen werden. 

* Anzahl der per default gepeicherten Checkpoints erhöhen. Statt 7 checkpoints nun bis zu 200 Checkpoints gespeichert. Anpassung in model_lib_v2, Zeile 443
* Anpassung, dass alle gespeicherten Checkpoints ausgewertet werden (und nicht nur einer), model_lib_v2 ,  Z.1135

nach: https://thachngoctran.medium.com/make-tensorflows-object-detection-validation-a-true-post-process-c45785f08d3a


Nach diesr Anpassung erfolgt die eigentliche Installation der Tensorflow-Object Detection API (davor noch setup.py Datei kopieren).

In [ ]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Nun ist die Installation der Tensorflow OD API abgeschlossen und kann mit dem *model_builder_tf2_test.py* Skript getestet werden. 

In [ ]:
!python /content/models/research/object_detection/builders/model_builder_tf2_test.py

### Vorbereitung

In [ ]:
%cd /content/
!wget https://raw.githubusercontent.com/MichaelSeitz98/Seminararbeit/main/labelmap.pbtxt

In [ ]:
labelmap_path = '/content/labelmap.pbtxt'

saved_model_dir = '/content/drive/MyDrive/training_effdetD0_8_20000_100/inference_graph/saved_model'
speicherPfadEvalBilder = '/content/drive/MyDrive/training_effdetD0_8_20000_100/angewendeteBilder_test'
schwellenwertanaye_speicherPfadEvalBilder = '/content/drive/MyDrive/training_effdetD0_8_20000_100/schwellenwertanalyse_test'

if (not os.path.isdir(schwellenwertanaye_speicherPfadEvalBilder)): 
    %mkdir {schwellenwertanaye_speicherPfadEvalBilder}
    print(f'Ordner erzeugt: {schwellenwertanaye_speicherPfadEvalBilder}')
else:
    print(f'Ordner: {schwellenwertanaye_speicherPfadEvalBilder} existiert schon')

input_bilder_pfad_val ='/content/drive/MyDrive/DS2_POOLG_80_20_20_1024px/test'

In [ ]:
import io
import os
import scipy.misc
import numpy as np
import six
import time
import glob
from IPython.display import display

from six import BytesIO

import matplotlib
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont

import tensorflow as tf
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

%matplotlib inline

In [ ]:
def load_image_into_numpy_array(path):
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(labelmap_path, use_display_name=True)
print(f'Catergory Index erzeugt aus LabelMap: {labelmap_path}')

Catergory Index erzeugt aus LabelMap: /content/labelmap.pbtxt


In [ ]:
tf.keras.backend.clear_session()
model = tf.saved_model.load(saved_model_dir)

print(f'Trainiertes Model erfolgreich geladen von: {saved_model_dir}')

Trainiertes Model erfolgreich geladen von: /content/drive/MyDrive/training_effdetD0_8_20000_100/inference_graph/saved_model


In [ ]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  input_tensor = tf.convert_to_tensor(image)
  input_tensor = input_tensor[tf.newaxis,...]

  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  if 'detection_masks' in output_dict:
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

## Schwellenwertanalyse starten


In [ ]:
bilderCounter = 0
danger = False
confident_score_child = 1.00

schwellenwerte = [1, 0.99, 0.95,0.90,0.85, 0.80, 0.75, 0.70, 0.60, 0.50, 0.40 ,0.30, 0.20, 0.10, 0.0]

for sw in schwellenwerte:
    print (f'Alle Bilder analysieren mit Schwellenwert: {sw}')
    SW_FOLDER = f'{schwellenwertanaye_speicherPfadEvalBilder}/SW_{sw}'
    OUTPUT_PATH_DANGER = f'{schwellenwertanaye_speicherPfadEvalBilder}/SW_{sw}/Gefahr'
    OUTPUT_PATH_NO_DANGER = f'{schwellenwertanaye_speicherPfadEvalBilder}/SW_{sw}/Keine_Gefahr'
    %mkdir {SW_FOLDER}
    %mkdir {OUTPUT_PATH_DANGER}
    %mkdir {OUTPUT_PATH_NO_DANGER}
    for image_path in glob.glob(f'{input_bilder_pfad_val}/*.jpg'):
      danger = False
      print(f'aktueller image_path:{image_path}')
      start_time_pro_bild = time.time()
      bilderCounter+=1
      image_name = os.path.basename(image_path).replace(".jpg", "")
      image_np = load_image_into_numpy_array(image_path)
      output_dict = run_inference_for_single_image(model, image_np)

      # Verarbeitungszeit
      classes_child = output_dict['detection_classes']
      scores_child = output_dict['detection_scores']
      boxes_child = output_dict["detection_boxes"]

      for i in range(0, len(classes_child)):
          if classes_child[i] == 1 and scores_child[i] >= sw:
                  danger = True
                  print ("Dangerous! Child detected!")

      outImg = vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          output_dict['detection_boxes'],
          output_dict['detection_classes'],
          output_dict['detection_scores'],
          category_index,
          instance_masks=output_dict.get('detection_masks_reframed', None),
          use_normalized_coordinates=True,
          min_score_thresh=sw,
          max_boxes_to_draw=15,
          line_thickness=3)
      im = Image.fromarray(outImg)
      
      #Auskommentiert für schneller Laufzeiten
      #display(Image.fromarray(image_np))
        
      if (danger):
            im.save(f'{OUTPUT_PATH_DANGER}/{image_name}_boxed_danger.jpg')
            print ('Hier gespeichert: ' + OUTPUT_PATH_DANGER + "/" + image_name + "_boxed.jpg")
      else:
            print('Keine Gefahr. Kein Kind entdeckt')
            im.save(f'{OUTPUT_PATH_NO_DANGER}/{image_name}_boxed_no_danger.jpg')
            print(f'gespeichert unter {OUTPUT_PATH_NO_DANGER}/{image_name}_boxed_no_danger.jpg')
    print(f'Fertig! Modell auf {bilderCounter} Bilder für Schwellenwert {sw} angewendet und unter {schwellenwertanaye_speicherPfadEvalBilder} gespeichert.')

print (f'Schwellenwertanalyse abgeschloseen für Schwellenwerte: {sw}')

Alle Bilder analysieren mit Schwellenwert: 0.99
aktueller image_path:/content/drive/MyDrive/DS2_POOLG_80_20_20_1024px/test/image (1).jpg
Keine Gefahr. Kein Kind entdeckt
gespeichert unter /content/drive/MyDrive/training_effdetD0_8_20000_100/schwellenwertanalyse_test/SW_0.99/Keine_Gefahr/image (1)_boxed_no_danger.jpg
aktueller image_path:/content/drive/MyDrive/DS2_POOLG_80_20_20_1024px/test/image (155).jpg
Keine Gefahr. Kein Kind entdeckt
gespeichert unter /content/drive/MyDrive/training_effdetD0_8_20000_100/schwellenwertanalyse_test/SW_0.99/Keine_Gefahr/image (155)_boxed_no_danger.jpg
aktueller image_path:/content/drive/MyDrive/DS2_POOLG_80_20_20_1024px/test/image (156).jpg
Keine Gefahr. Kein Kind entdeckt
gespeichert unter /content/drive/MyDrive/training_effdetD0_8_20000_100/schwellenwertanalyse_test/SW_0.99/Keine_Gefahr/image (156)_boxed_no_danger.jpg
aktueller image_path:/content/drive/MyDrive/DS2_POOLG_80_20_20_1024px/test/image (157).jpg
Keine Gefahr. Kein Kind entdeckt
gespeicher